In [9]:
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Grayscale, Gaussian blur, Otsu's threshold
image = cv2.imread('Data_EV_1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Morph open to remove noise and invert image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
invert = 255 - opening

# Perform text extraction
text = pytesseract.image_to_string(invert, lang='eng', config='--psm 6')
print(text)

#cv2.waitKey()

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [6]:
text1=text
text1
text2="| 12:17 4 ~ Fe }\n< My Trips Trip G)\n28 Mar 2024 Long Trip\nTime Summary\n01 h09m00 s\nDistance\n53.9mile\nTravel Time Max. Speed\n01 109122. 80. 4imph\nStopped Time Avg Speed\n00.29. 46 5Sinpn\nMax. Altitude Avg Altitude\n1565: 9418\nMap & Graphs\n%y %\n"

NameError: name 'text' is not defined

In [55]:
import re
# Regex patterns to extract relevant data
patterns = {
    "Time Summary": r"Time Summary\s*([\d\w\s]+?)(?=\n|$)",
    "Distance": r"Distance\s*([\d\.]+mile)",
    "Avg Speed": r"Avg Speed\s*([\d\.]+\s*[a-zA-Z]+)",
    "Max Altitude": r"Avg Altitude\s*([\d,]+)",
    "Avg Speed": r"Avg Speed\s*([\d\w\s:.]+?)(?=\n|$)",
    "Avg Altitude": r"Avg Altitude\s*([\d,]+)"
}

# Extracting and cleaning the data
extracted_data = {}
for key, pattern in patterns.items():
    # Search using the pattern
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        value = match.group(1).strip()
        if key == "Avg Speed":
                # Remove any unwanted prefix numbers followed by a period
                value = re.sub(r"^\d+\.\d+\s*", "", value)
                value= value.replace("Sinpn","mph")
                value= value.replace(" ",".")
                value= value.replace("..","")
        if key in ["Max Altitude", "Avg Altitude"]:
            value = value.replace(":", "").strip() + " ft"  # Correct format and OCR error ':'
        extracted_data[key] = value
pattern = r"Avg Altitude\s*([\d,]+)\s*:\s*([\d,]+)"

# Search using the pattern
match = re.search(pattern, ocr_text, re.IGNORECASE)
if match:
    avg_altitude_first = match.group(1).replace(',', '').strip()   # Clean up first number
    avg_altitude_next = match.group(2).replace(',', '').strip()  # Clean up second number

# # Display the cleaned data
extracted_data['Avg Altitude']= avg_altitude_next + " ft"
extracted_data["Elevation Gain"]= str((int(avg_altitude_next) - int(avg_altitude_first) ))+ " ft"
# patterns_2
extracted_data
# for key, pattern in patterns_2.items():
#     # Search using the pattern
#     match = re.search(pattern, ocr_text, re.IGNORECASE)
#     extracted_data[2+key]

{'Time Summary': '01 h09m51 s',
 'Distance': '53.8mile',
 'Avg Speed': '46.5mph',
 'Max Altitude': '1565 ft',
 'Avg Altitude': '9418 ft',
 'Elevation Gain': '7853 ft'}

In [60]:
import re
import pandas as pd

# Initialize DataFrame to store data
data_columns = ["Time Summary", "Distance", "Avg Speed", "Max Altitude", "Avg Altitude", "Elevation Gain"]
extracted_data_df = pd.DataFrame(columns=data_columns)

# Function to process text
def process_text(ocr_text):
    # Regex patterns to extract relevant data
    patterns = {
        "Time Summary": r"Time Summary\s*([\d\w\s]+?)(?=\n|$)",
        "Distance": r"Distance\s*([\d\.]+mile)",
        "Avg Speed": r"Avg Speed\s*([\d\.]+\s*[a-zA-Z]+)",
        "Max Altitude": r"Avg Altitude\s*([\d,]+)",
        "Avg Speed": r"Avg Speed\s*([\d\w\s:.]+?)(?=\n|$)",
        "Avg Altitude": r"Avg Altitude\s*([\d,]+)"
    }

    # Extracting and cleaning the data
    extracted_data = {}
    for key, pattern in patterns.items():
        # Search using the pattern
        match = re.search(pattern, ocr_text, re.IGNORECASE)
        if match:
            value = match.group(1).strip()
            if key == "Avg Speed":
                value = re.sub(r"^\d+\.\d+\s*", "", value)  # Remove unwanted prefix numbers
                value = value.replace("Sinpn", "mph").replace(" ", ".").replace("..", "")
            if key in ["Max Altitude", "Avg Altitude"]:
                value = value.replace(":", "").strip() + " ft"  # Correct format and OCR error
            extracted_data[key] = value

    # Further process for elevation gain
    pattern = r"Avg Altitude\s*([\d,]+)\s*:\s*([\d,]+)"
    match = re.search(pattern, ocr_text, re.IGNORECASE)
    if match:
        avg_altitude_first = match.group(1).replace(',', '').strip()
        avg_altitude_next = match.group(2).replace(',', '').strip()
        extracted_data['Avg Altitude'] = avg_altitude_next + " ft"
        extracted_data["Elevation Gain"] = str(int(avg_altitude_next.replace(" ft", "")) - int(avg_altitude_first.replace(" ft", ""))) + " ft"

    return extracted_data

# Example texts list
texts = [text1,text2]  # You can add more OCR texts to this list

# Process each text and append results to DataFrame
for text in texts:
    data = process_text(text)
    extracted_data_df = extracted_data_df.append(data, ignore_index=True)

# Show the DataFrame
print(extracted_data_df)


  Time Summary  Distance Avg Speed Max Altitude Avg Altitude Elevation Gain
0  01 h09m51 s  53.8mile   46.5mph      1565 ft      9418 ft        7853 ft
1  01 h09m00 s  53.9mile   46.5mph      1565 ft      9418 ft        7853 ft


C:\Users\msikh\AppData\Local\Temp\ipykernel_25624\3634173401.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data_df = extracted_data_df.append(data, ignore_index=True)
C:\Users\msikh\AppData\Local\Temp\ipykernel_25624\3634173401.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data_df = extracted_data_df.append(data, ignore_index=True)
